<b><font color="SteelBlue" size="+3">Implémentez un modèle de scoring</font></b>

# Introduction

Ce note book est la suite de l'EDA, il va permettre de présenter le prétraitement des données, le feature engineering et la modélisation.

In [1]:
# Chargement des librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# classe de feature engineering perso
from feature_pipeline import FeatureEngineeringPipeline

# Feature engineering

In [2]:
# Initialiser le pipeline
pipeline = FeatureEngineeringPipeline()

In [3]:
# Charger les données d'entraînement
train_df = pd.read_csv('data/sources/application_train.csv')

In [14]:
train_df.shape

(307511, 122)

In [4]:
# Entraîner le pipeline
train_df_transformed = pipeline.fit(train_df)

Bureau and bureau_balance data - done in 17s
previous_application - done in 24s
previous applications balances - done in 670s


In [5]:
# Sauvegarder les paramètres de transformation
pipeline.save('param')

In [7]:
train_df_transformed.shape

(307506, 734)

In [9]:
train_df_transformed.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.374558,0.374558,2.149501,0.467976,1.175716,1.175716,0.064052,0.092379,0.951334,1.372057
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.584212,0.186646,15.710334,1.813930,0.290662,0.239484,0.077841,0.083552,1.098485,1.179082
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.793667,0.793667,1.566513,1.051282,0.187652,0.187652,0.042791,0.045521,3.622222,3.853333
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.735762,0.083638,23.303080,2.120394,2.643133,1.497703,0.018185,NaN,0.113722,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.561561,0.000008,1.037195,0.558600,0.931669,0.538926,0.061350,0.057646,0.402513,0.378209


In [13]:
train_df_transformed.dtypes.value_counts()

float64    676
int64       42
object      16
Name: count, dtype: int64

In [15]:
def unique_values_per_object_column(df):    # Sélectionner les colonnes de type object
    object_columns = df.select_dtypes(include=['object']).columns
    
    # Créer un dictionnaire pour stocker le nombre de valeurs uniques par colonne
    unique_values_counts = {}
    
    # Boucle sur les colonnes object et compter les valeurs uniques
    for col in object_columns:
        unique_values_counts[col] = df[col].nunique()
    
    # Afficher les résultats
    for col, count in unique_values_counts.items():
        print(f"{col}: {count} valeurs uniques")

unique_values_per_object_column(train_df_transformed)


NAME_CONTRACT_TYPE: 2 valeurs uniques
CODE_GENDER: 2 valeurs uniques
FLAG_OWN_CAR: 2 valeurs uniques
FLAG_OWN_REALTY: 2 valeurs uniques
NAME_TYPE_SUITE: 7 valeurs uniques
NAME_INCOME_TYPE: 8 valeurs uniques
NAME_EDUCATION_TYPE: 5 valeurs uniques
NAME_FAMILY_STATUS: 6 valeurs uniques
NAME_HOUSING_TYPE: 6 valeurs uniques
OCCUPATION_TYPE: 18 valeurs uniques
WEEKDAY_APPR_PROCESS_START: 7 valeurs uniques
ORGANIZATION_TYPE: 58 valeurs uniques
FONDKAPREMONT_MODE: 4 valeurs uniques
HOUSETYPE_MODE: 3 valeurs uniques
WALLSMATERIAL_MODE: 7 valeurs uniques
EMERGENCYSTATE_MODE: 2 valeurs uniques


In [ ]:
# Charger les paramètres de transformation
pipeline.load('path/to/save/directory')

# Charger les données de test
test_df = pd.read_csv('path/to/test.csv')

# Transformer les données de test
test_df_transformed = pipeline.transform(test_df)

# Maintenant, vous pouvez utiliser `test_df_transformed` pour la prédiction